# Семинар : Градиентный спуск

## Вступление

Градиентный спуск - это алгоритм оптимизации, т.е. численный метод поиска локального минимума/максимума функции. 

### План семинара

1. Введение
2. Суть метода градиентного спуска
3. Модификации градиентного спуска
    

## 1. Введение


##### Наводящие вопросы из геометрии:
*  Каким уравнением задается прямая на плоскости? Чем отличаются записи $y = kx + b$ и $ax + by + c = 0$?

* Запишите уравнение плоскости в трехмерном пространстве, уравнение гиперплоскости в многомерном пространстве. Используя введенные обозначения, ответьте, в пространстве какой размерности задается гиперплоскость из предыдущего вопроса?

* Если какой-то из коэффициентов в уравнении гиперплоскости равен 0, что это геометрически означает?

* Что означает, что свободный член равен 0?

### Визуализация функции от двух переменных

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
#from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

Ниже приведена функция, визуализирующая поверхности. 

In [ ]:
def plot_3d(fun, a=-1, b=1, c=-1, d=1, trace=None): #двумерная пов-ть в трехмерном пр-ве
    """
    Визуализирует функцию fun на квадрате [a, b] x [c, d]
    fun : функция, принимающая два аргумента
         (np.array одинакового размера) и возвращающая
          np.array того же размера со значениями функции
          в соответствующих точках
    Дополнительно можно нарисовать ломаную линию из N точек,
    лежащую на получившейся поверхности
    trace : np.array размера N x 2 - координаты на плоскости,
            обозначающие точки ломаной
    """
    fig = plt.figure(figsize=(10, 8))

    # Make grid
    x1_ = np.linspace(a, b, 100)
    x2_ = np.linspace(c, d, 100)
    x1, x2 = np.meshgrid(x1_, x2_)
    y = fun(x1, x2)

    # Plot the surface
    ax = fig.add_subplot(1, 1, 1, projection="3d")
    ax.plot_surface(x1, x2, y, alpha=0.6)
    ax.contour(x1, x2, y, zdir="z", offset=y.min(), cmap=cm.coolwarm)
    plt.xlabel("x")
    plt.ylabel("y")
    
    # Plot 3d line
    if trace is not None:
        y_trace = fun(trace[:, 0], trace[:, 1])
        ax.plot(trace[:, 0], trace[:, 1], y_trace, "o-")
        ax.set_xlim(x1.min(), x1.max())
        ax.set_ylim(x2.min(), x2.max())
        ax.set_zlim(y.min(), y.max())

In [ ]:
#print(not None)



Нарисуем с ее помощью параболоид:

$$z=x^2+y^2$$

In [ ]:
fun = lambda x1, x2: x1**2 + x2**2
plot_3d(fun)

Круги на плоскости показывают __проекции линий уровня__ параболоида.

**Задание**: нарисуйте плоскость $y = x_1 + 2 x_2 + 3$:

In [ ]:
fun = lambda x1, x2: x1 + 2*x2 + 3 #- x1 - x2 + 1
# fun = # your code
plot_3d(fun)

**Задание**: нарисуйте плоскость, параллельную любой из горизонтальных осей:

In [ ]:
fun = lambda x1, x2: x1+1 #0*x1+0*x2+1 #параллельно y
# fun = # your code
plot_3d(fun)

**Задание**: нарисуйте плоскость, проходящую через начало координат:

In [ ]:
fun = lambda x1, x2: x1 + x2 
# fun = # your code
plot_3d(fun)

## 2. Суть метода градиентного спуска

### Градиентный спуск, теоретическая часть

Градиент функции $f(x) = f(x_1, \dots, x_d)$ от многих переменных в точке $x_0$ - это вектор ее частных производных, вычисленных в точке $x_0$.
$$\nabla_x f \bigl | _{x_0} = \biggl(\frac{\partial f}{\partial x_1}, \dots, \frac{\partial f}{\partial x_d} \biggr ) \biggl | _{x_0}$$

Разберем два простых примера вычисления градиента в случае функции от двух переменных.

#### Задача 1

Найдите градиент линейной функции $f(x) = f(x_1, x_2) = c_1 x_1 + c_2 x_2$ ($c_1$ и $c_2$ - фиксированные числа). 

__Решение.__

Найдем первую частную производную: 

$$\frac{\partial f}{\partial x_1} = \frac{\partial (c_1 x_1 + c_2 x_2)}{\partial x_1} = c_1.$$

Значит, первая компонента градиента равна $c_1$. Аналогично со второй компонентой. 

Ответ:

$$\nabla_x f = (c_1, c_2)$$

Можно подставить конкретные коэффициенты, например $c_1 = 3$ и $c_2 = 7$. Тогда градиент будет равен $(3, 7)$. 

#### Задача 2
Найдите градиент квадратичной функции $f(x) = f(x_1, x_2) = c_1 x_1^2 + c_2 x_2^2$ ($c_1$ и $c_2$ - фиксированные числа). 

__Решение.__
Найдем первую частную производную: 

$$\frac{\partial f}{\partial x_1} = \frac{\partial (c_1 x_1^2 + c_2 x_2^2)}{\partial x_1} = 2 c_1 x_1.$$

Значит, первая компонента градиента равна $2 c_1 x_1$. Аналогично со второй компонентой. 

Ответ:

$$\nabla_x f = (2 c_1 x_1, 2 c_2 x_2)$$

При неотрицательных $c_1, c_2$ минимум такой квадратичной функции достигается в 0.
Наша следующая цель - найти этот минимум с помощью градиентного спуска. 

##### Вопросы:
* Какую (оптимизационную) задачу решает градиентный спуск?
* Как работает алгоритм градиентного спуска?
* Как выбирать начальную инициализацию в градиентном спуске?
* Когда останавливать градиентный спуск?

### Градиент квадратичной функции

Воспользуемся кодом для вычисления квадратичной функции  и обобщим ее на случай произвольных коэффициентов:

In [ ]:
def fun(x1, x2, c1=1, c2=1):
    return c1 * (x1**2) + c2 * (x2**2)

Эта функция способна обрабатывать x1 и x2 любой размерности, но градиенты будем считать в предположении, что x1 и x2 - числа 

Реализуем подсчет градиента функции fun. Код функции вычисления градиента в одной точке (согласно описанию):

In [ ]:
def grad_fun(x1, x2, c1=1, c2=1):
    """
    Функция берет 2 числа, обозначающую точку, в которой вычисляется градиент,
    и возвращает np.array размера (2,) - градиент квадратичной функции
    Опциональные аргументы: c1 и c2 - коэффициенты
    """
    return np.array([2*c1*x1, 2*c2*x2])
        
    #    pass

Проверим:

In [ ]:
#ответ array([2., 9.])
grad_fun(x1=0.5, x2=1.5, c1=2, c2=3)

### Градиентный спуск (GD), реализация

__Реализуем градиентный спуск__. 
Он работает следующим образом: 
1. сначала инициализируется начальная точка x 
1. затем повторяются итерации (общая формула для минимизации $f$):
$$x^{(t)} = x^{(t-1)} - \alpha \nabla_{x^{(t-1)}} f$$
Здесь $\alpha$ - длина шага.


In [ ]:
def grad_descend(grad_fun, step_size=0.1, num_steps=50):
    """
    Реализует градиентный спуск
    Аргументы:
    * grad_fun - функция, вычисляющая градиент
    * step_size - длина шага
    * num_steps - число итераций

    Возвращает np.array размера (num_steps+1) x 2,
    (i+1)-й элемент - точка на (i+1)-й итерации,
    нулевой элемент - случайная инициализация
    """
    #массив чисел из равн. распр. в диапазоне [0, 1) размера (2,)
    x = np.random.rand(2) * 4 - 2
    
    trace = np.zeros((num_steps + 1, 2))
    trace[0] = x        #начальное приближение
    for it in range(num_steps):
        grad = grad_fun(x[0], x[1]) #градиент
        
        x = x - step_size * grad #градиентный шаг
        
        trace[it + 1] = x #приближение к min на шаге it + 1
    return trace        
    
#    pass

Протестируем функцию (последний элемент должен быть близок к 0):

In [ ]:
np.random.seed(16)

trace = grad_descend(grad_fun)
trace

__Визуализируем градиентный спуск__. Для этого передадим нашу траекторию оптимизации в качестве последнего аргумента функции plot_3d.

In [ ]:
trace = grad_descend(grad_fun, 0.1, 30)
print(trace[0])
print(trace[30])
plot_3d(fun, trace=trace)

Запустим оптимизацию несколько раз, чтобы посмотреть, __как ведет себя процесс при различных случайных начальных приближениях__ (вообще говоря, сходимость градиентного спуска зависит от выбора начального приближения):

In [ ]:
for i in range(3):
    trace = grad_descend(grad_fun, 0.1, 30)
    plot_3d(fun, trace=trace)
    print(trace[0])
    print(trace[30])
    print()

__Используем разную длину шага__ из множества $(0.01, 0.1, 0.5, 1)$.

In [ ]:
for ss in [0.01, 0.1, 0.5, 1]:
    np.random.seed(0)
    trace = grad_descend(grad_fun, step_size=ss)
    plot_3d(fun, trace=trace)
   

При маленькой длине шага процесс идет слишком медленно, при большой - может разойтись.

Наконец, __попробуем использовать другие коэффициенты квадратичной функции__. 
Оптимизируем функцию $f(x) = x_1^2 + 5 x_2^2$, пробуя длину шага (0.1, 0.2, 0.5):

In [ ]:
fun_c = lambda x1, x2: fun(x1, x2, 1, 5)
grad_fun_c = lambda x1, x2: grad_fun(x1, x2, 1, 5)

for ss in [0.1, 0.2, 0.5]:
    trace = grad_descend(grad_fun_c, step_size=ss)
    plot_3d(fun_c, trace=trace)

"Вытянутую" функцию сложнее оптимизировать. Именно поэтому __данные рекомендуется масштабировать__ (нормировать) перед обучением модели, чтобы избежать таких колебаний при оптимизации.

###  Градиентый спуск для оценки параметров линейной регрессии 
На простом примере разберём основные тонкости.

__Сгенерируем матрицу объекты-признаки $X$ и вектор весов $w_{true}$__, __вектор целевых переменных $y$__ будем вычислять как:

$$
y = Xw_{true} + \epsilon,
$$
где $\epsilon \sim N(0, 1)$ (нормальный шум).

In [ ]:
np.random.seed(25)

In [ ]:
#генерируем данные (сгенерируем линейную зависимость с шумом)
n_features = 2
n_objects = 300 
batch_size = 10
num_steps = 43

w_true = np.random.normal(size=(n_features,)) #вектор весов w_true  #normal(loc=0.0, scale=1.0, size) 

X = np.random.uniform(-5, 5, (n_objects, n_features)) #uniform(a, b, shape)
X *= (np.arange(n_features) * 2 + 1)[np.newaxis, :] #поэлементное умножение, X[n_objects, n_features] - матр. объект-признак
Y = X.dot(w_true) + np.random.normal(0, 1, (n_objects)) #ответы y=Xw_true+eps #normal(loc, scale, size)
w_0 = np.random.uniform(-2, 2, (n_features)) #начальное приближение для весов - массив np.array (n_features,) 

In [ ]:
w_true

In [ ]:
w_0

Напомним, что
$Q(a) = \frac 1 \ell \sum_{i=1}^\ell L(y_i, a(x_i))$ - функционал ошибки, или в линейной регрессии Mean Squared Error (MSE):
$$Q (a, X) = \frac{1}{l}\sum^l_{i=1}(a(x_i) - y_i)^2,$$
где $a(x)=  w_0+w_1x_1+...w_dx_d$ - приближение (алгоритм), или
$$Q(w)=
\frac{1}{l}\sum_{i=1}^{l}(\langle w,x_i \rangle -y_i)^2\rightarrow min_{w_0,w_1,...,w_d}
$$

где $x_i$ — это $i$-ый объект датасета, $y_i$ — правильный ответ для $i$-го объекта, а $w$ — веса нашей линейной модели.

Мы решаем __задачу минимизации функционала__ $Q$, т.е. 
$$Q(w)=
\frac{1}{l}\sum_{i=1}^{l}(\langle w,x_i \rangle -y_i)^2\rightarrow min_{w_0,w_1,...,w_d}
$$
Если мы попробуем минимизировать $Q$ "в лоб", то __оптимальный набор параметров__ будет выглядеть так: 
$$w = (X^TX)^{-1}X^Ty$$ В этой формуле присутствует обращение матрицы $X^TX$ — очень трудоёмкая операция при большом количестве признаков. Сложность вычислений в этом случае равна $O(d^3 + d^2 \ell)$, где $l$ - число объектов, $d$ — число признаков. При решении задач такая трудоёмкость часто оказывается непозволительной, поэтому параметры ищут итерационными методами, стоимость которых меньше.   
Один из них — __градиентный спуск__.

В __градиентном спуске__ значения параметров на следующем шаге получаются из значений параметров на текущем шаге смещением в сторону антиградиента функционала $Q(w)$: 

$$w^{(t+1)} = w^{(t)} - \eta_t \nabla Q(w^{(t)}),$$
где $\eta_t$ — длина шага градиентного спуска, $\nabla_w Q(w) =\frac{1}{l}\sum_{i=1}^{l}\nabla_w L_i(w)$.

Функционал MSE в матричном виде можно записать так:
$$
Q(w) =\frac{1}{l} (y - Xw)^T(y-Xw)
$$
или
$$
Q(w, X, y) = \frac{1}{l} || Xw - y ||^2,
$$
где $X$ — это матрица объекты-признаки, а $y$ — вектор правильных ответов.
А соответствующий градиент:
$$
\nabla_w Q(w) =\frac{1}{l} \nabla_w[y^Ty - y^TXw - w^TX^Ty + w^TX^TXw] =\frac{1}{l}( 0 - X^Ty - X^Ty + (X^TX + X^TX)w) = \frac{2}{l}X^T(Xw-y), т.е.
$$
 
$$
\nabla_w Q(w) =\frac{2}{l}X^T(Xw-y)
$$

Сложность вычислений при нахождении параметров в данном случае равна $O(d \ell)$. 

__Обучим на полученных данных линейную регрессию для MSE при помощи полного градиентного спуска__.  
В результате, получим вектор оценок параметров

In [ ]:
w = w_0.copy() #копия нач. приближения весов
w_list = [w.copy()] #список массивов np.array для весов
step_size = 1e-2

for i in range(num_steps):
    #градиентный шаг:
    w -= 2 * step_size * np.dot(X.T, np.dot(X, w) - Y) / Y.shape[0] #/ Y.shape[0] - если в Q есть деление на l
    w_list.append(w.copy()) #заполняем список массивов np.array для весов
w_list = np.array(w_list) #преобразуем список в массив np.array для весов (num_steps,n_features)
w_list

In [ ]:
w_true

In [ ]:
#Применение линейной регрессии 
from sklearn import linear_model

#from sklearn.lenear_model import LinearRegressin #МНК (w по точной формуле) 
#from sklearn.lenear_model import SGDRegressor #градиентный спуск

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X, Y)
print("Coefficients: \n", regr.coef_)

### Визуализация траекторий GD

__Покажем последовательность оценок параметров $w^{(t)}$, получаемых в ходе итераций__. Красная точка — $w_{true}$.

Для этого создадим функцию, которую будем и дальше использовать для визуализации процесса оптимизации.

In [ ]:
def plot_gradient(w_list, title):
    A, B = np.meshgrid(np.linspace(-2, 2, 100), np.linspace(-2, 2, 100)) #создаем сетку

    levels = np.empty_like(A) #линии уровня (массив той же формы и типа, что A)
    #для каждой точки сетки считаем Q
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            w_tmp = np.array([A[i, j], B[i, j]])
            levels[i, j] = np.mean(np.power(np.dot(X, w_tmp) - Y, 2)) #значение (линия уровня Q)

    plt.figure(figsize=(15, 6))
    plt.title(title)
    plt.xlabel(r"$w_1$")
    plt.ylabel(r"$w_2$")
    plt.xlim((w_list[:, 0].min() - 0.2, w_list[:, 0].max() + 0.2)) #диапозон по x
    plt.ylim((w_list[:, 1].min() - 0.2, w_list[:, 1].max() + 0.2)) #диапозон по y
    plt.gca().set_aspect("equal") #оси

    # visualize the level set
    CS = plt.contour(
        A, B, levels, levels=np.logspace(0, 1, num=20), cmap=plt.cm.rainbow_r
    ) #линии уровня #np.logspace(0, 1, num=20) от 10^0 до 10^1 - 20 чисел 
    CB = plt.colorbar(CS, shrink=0.8, extend="both") #шкала

    # visualize trajectory
    plt.scatter(w_true[0], w_true[1], c="r") #истинные веса (красная точка)
    plt.scatter(w_list[:, 0], w_list[:, 1])  #веса, полученные в ходе градиентного спуска - синие точки, их число num_steps
    plt.plot(w_list[:, 0], w_list[:, 1])   #соединение точек синими отрезками

    plt.show()

$$w^{(t+1)} = w^{(t)} - \eta_t \nabla Q(w^{(t)}),$$
где $\eta_t=const$ — длина шага градиентного спуска, $\nabla_w Q(w) =\frac{1}{l}\sum_{i=1}^{l}\nabla_w L_i(w)$.

In [ ]:
plot_gradient(w_list, "GD trajectory")

Из мат. анализа известно, градиент перпендикулярен линиям уровня. Это объясняет такие зигзагообразные траектории градиентного спуска. Для большей наглядности в каждой точке пространства __посчитаем градиент функционала и покажем его направление__.

In [ ]:
# compute level set
A, B = np.meshgrid(np.linspace(-3, 3, 100), np.linspace(-3, 3, 100)) #создаем сетку
A_mini, B_mini = np.meshgrid(np.linspace(-3, 3, 20), np.linspace(-2, 2, 27))

levels = np.empty_like(A)
#для каждой точки сетки считаем Q
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        w_tmp = np.array([A[i, j], B[i, j]])
        levels[i, j] = np.mean(np.power(np.dot(X, w_tmp) - Y, 2)) #значение (линия уровня Q)

# visualize the level set
plt.figure(figsize=(13, 9))
CS = plt.contour(
    A, B, levels, levels=np.logspace(-1, 1.5, num=30), cmap=plt.cm.rainbow_r
)                                                #линии уровня
CB = plt.colorbar(CS, shrink=0.8, extend="both") #шкала

# visualize the gradients
gradients = np.empty_like(A_mini)
for i in range(A_mini.shape[0]):
    for j in range(A_mini.shape[1]):
        w_tmp = np.array([A_mini[i, j], B_mini[i, j]])
        antigrad = - 2 * 1e-3 * np.dot(X.T, np.dot(X, w_tmp) - Y) / Y.shape[0] #антиград. (с "-"), 1e-3 - связь с дл.вектора
        plt.arrow(A_mini[i, j], B_mini[i, j], antigrad[0], antigrad[1], head_width=0.02) #стрелки

plt.title("Antigradients demonstration")
plt.xlabel(r"$w_1$")
plt.ylabel(r"$w_2$")
plt.xlim((w_true[0] - 1.5, w_true[0] + 1.5))
plt.ylim((w_true[1] - 0.5, w_true[1] + 0.7))
plt.gca().set_aspect("equal")
plt.show()

## 3. Модификации градиентного спуска

### Стохастический градиентый спуск (SGD)
Существуют различные модификации градиентного спуска. 
Мы познакомимся с одной из них — стохастическим градиентным спуском.
От варианта выше он отличается заменой градиента на несмещённую оценку (это точечная оценка, математическое ожидание которой равно оцениваемому параметру) градиента по одному или нескольким объектам. В этом случае сложность становится $O(dk)$, где $k$ — количество объектов, по которым оценивается градиент, $k \ll \ell$. Для больших массивов данных стохастический градиентный спуск может дать значительное преимущество в скорости по сравнению со стандартным градиентным спуском.

#### Применяем

Модифицируем код и будем считать градиент не по всему набору данных, а только __по случайной подвыборке объектов batch__ (в частности, по одному случайному объекту). Получим траектории стохастического градиентного спуска.


$$w^{(t+1)} = w^{(t)} - \eta_t \frac{1}{k}\sum_{j=1}^{k}\nabla_w L_j(w^{(t)}),$$
где $\eta_t=const$ — длина шага градиентного спуска, $j$ - случайно выбранные номера слагаемых из функционала $Q$.


__Функция генерации выборки и подсчета градиента__:

In [ ]:
def calc_grad_on_batch(X, Y, w, batch_size):
    #создаем набор индексов для batch_size
    sample = np.random.randint(n_objects, size=batch_size) ##равн. расп. (цел) от 0 до n_objects) размера size
    return 2 * np.dot(X[sample].T, np.dot(X[sample], w) - Y[sample]) / batch_size




In [ ]:
print(n_features)
print(n_objects) 
print(batch_size)
print(num_steps)

__Градиентный шаг:__

In [ ]:
step_size = 1e-2

w = w_0.copy() #начальное приближение весов
w_list = [w.copy()]

for i in range(num_steps):
    w -= step_size * calc_grad_on_batch(X, Y, w, batch_size) #градиентный шаг на batch_size, а не на всей выборке
    w_list.append(w.copy()) 

w_list = np.array(w_list) #массив наборов весов (num_steps*2) 

 ### Визуализация траекторий SGD

In [ ]:
plot_gradient(w_list, "SGD trajectory")

Как видно, метод стохастического градиента «бродит» вокруг оптимума. На такое поведение можно повлиять подбором шага градиентного спуска $\eta_t$. 

### Экспериментируем с градиентным спуском.

### ГРАДИЕНТНЫЙ ШАГ

__Будем уменьшать шаг градиентного спуска на каждой итерации__.

$$\eta_t=\frac{s_{0}}{t+1}$$

In [ ]:
step_size_0 = 0.01

w = w_0.copy()
w_list = [w.copy()]


for i in range(num_steps):
    step_size = step_size_0 / (i + 1) #уменьшаем шаг
    w -= step_size * calc_grad_on_batch(X, Y, w, batch_size) #градиентный шаг на batch_size
    w_list.append(w.copy())

w_list = np.array(w_list)

In [ ]:
plot_gradient(w_list, "SGD trajectory with dynamic lr") #lr - learning rate

Теперь градиентный спуск движется более направленно, но не доходит до оптимума.   
__Более сложная схему изменения длины шага__:
$$
    \eta_t
    =
    \lambda
    \left(
        \frac{s_0}{s_0 + t}
    \right)^p.
$$
Возьмем $s_0 = 1$ и поэкспериментируем с разными $\lambda$ и $p$.

__Функция градиентного шага с изменяемой длиной шага:__

In [ ]:
def sgd_with_lr_schedule(lambda_param, p=0.5, s_init=1.0, batch_size=10):
    w = w_0.copy()
    w_list = [w.copy()]

    for i in range(num_steps):
        step_size = lambda_param * np.power(s_init / (s_init + i), p)
        w -= step_size * calc_grad_on_batch(X, Y, w, batch_size)
        w_list.append(w.copy())

    return np.array(w_list)

In [ ]:
w_list = sgd_with_lr_schedule(lambda_param=0.01, p=0.8)
plot_gradient(w_list, f"SGD with lr shedule, lambda={0.01}, p={0.8}")

In [ ]:
w_list = sgd_with_lr_schedule(lambda_param=0.01, p=0.5)
plot_gradient(w_list, f"SGD with lr shedule, lambda={0.01}, p={0.5}")

In [ ]:
w_list = sgd_with_lr_schedule(lambda_param=0.01, p=0.1)
plot_gradient(w_list, f"SGD with lr shedule, lambda={0.01}, p={0.1}")

Коэффициенты в формуле для длины шага являются гиперпараметрами, и их нужно подбирать. Желательно использовать для этого валидационную выборку.

### РАЗМЕР ПОДВЫБОРКИ

__Посмотрим, как размер подвыборки, по которой оценивается градиент, влияет на сходимость__.

In [ ]:
w_list = sgd_with_lr_schedule(lambda_param=0.01, p=0.35, batch_size=10)
plot_gradient(w_list, f"SGD with batch_size = {10}")

In [ ]:
w_list = sgd_with_lr_schedule(lambda_param=0.01, p=0.35, batch_size=100)
plot_gradient(w_list, f"SGD with batch_size = {100}")

Вывод: чем больше размер подвыборки, тем более стабильная траектория градиентного спуска.

### Сравнение скоростей сходимости

Теперь посмотрим, насколько быстро достигают оптимума методы полного и стохастического градиентного спуска. Сгенерируем выборку и построим два графика: 
 - зависимость значения функционала ошибки (лосса) от номера итерации (шага) градиентного спуска;
 - зависимость нормы градиента функционала ошибки от номера итерации (шага) градиентного спуска.


 Норма градиента функционала ошибки вычисляется по формуле:

 $$
||\nabla Q(w)|| = ||\frac{2}{l}X^T(Xw-Y)||
 $$

In [ ]:
#генерируем новые данные
n_features = 50
n_objects = 1000
num_steps = 200
batch_size = 10

#истинные веса
w_true = np.random.uniform(-2, 2, n_features) #равном.распр.(low=0.0, high=1.0, size=None)

X = np.random.uniform(-10, 10, (n_objects, n_features)) #матр. объект-признак
Y = X.dot(w_true) + np.random.normal(0, 5, n_objects) #ответы - normal(loc, scale, size)

In [ ]:
from scipy.linalg import norm  #для вычисления нормы матрицы

step_size_sgd = 1e-2
step_size_gd = 1e-2
w_sgd = np.random.uniform(-4, 4, n_features)
w_gd = w_sgd.copy()
residuals_sgd = [np.mean(np.power(np.dot(X, w_sgd) - Y, 2))]  #список для ошибок (sgd)
residuals_gd = [np.mean(np.power(np.dot(X, w_gd) - Y, 2))]   #список для ошибок (gd)

norm_sgd = [] #список для норм градиента (sgd)
norm_gd = []  #список для норм градиента (gd)


for i in range(num_steps):
    step_size = step_size_sgd / ((i + 1) ** 0.51) #градиентный шаг (уменьшаем)
    sample = np.random.randint(n_objects, size=batch_size) #индексы случайной подвыборки batch_size элементов 

    w_sgd -= (
        2
        * step_size
        * np.dot(X[sample].T, np.dot(X[sample], w_sgd) - Y[sample])
        / batch_size
    )      #веса sgd
    residuals_sgd.append(np.mean(np.power(np.dot(X, w_sgd) - Y, 2))) #добавляем в список ошибку sgd 
    norm_sgd.append(norm(np.dot(X[sample].T, np.dot(X[sample], w_sgd) - Y[sample]))) #добавляем норму градиента (sgd) (без const)

    w_gd -= 2 * step_size_gd * np.dot(X.T, np.dot(X, w_gd) - Y) / Y.shape[0] #веса gd
    residuals_gd.append(np.mean(np.power(np.dot(X, w_gd) - Y, 2))) #добавляем в список ошибку gd
    norm_gd.append(norm(np.dot(X.T, np.dot(X, w_gd) - Y)))  #добавляем норму градиента (gd)   (без const)

In [ ]:
plt.figure(figsize=(13, 6))
plt.plot(range(num_steps + 1), residuals_gd, label="Full GD")
plt.plot(range(num_steps + 1), residuals_sgd, label="SGD")

plt.title("Значение функционала ошибки (функции потерь) в зависимости от номера итерации (шага)")
plt.xlim((-1, num_steps + 1))
plt.legend()
plt.xlabel("Номер итерации")
plt.ylabel(r"Q($w$)")
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(13, 6))
plt.plot(range(num_steps), norm_gd, label="Full GD")
plt.plot(range(num_steps), norm_sgd, label="SGD")

plt.title("Норма градиента функционала ошибки (функции потерь) в зависимости от номера итерации (шага))")
plt.xlim((-1, num_steps + 1))
plt.legend()
plt.xlabel("Номер итерации")
plt.ylabel(r"$||\nabla Q$($w$)||")
plt.grid()
plt.show()

Как видно, GD буквально за несколько итераций оказывается вблизи оптимума, в то время как поведение SGD может быть весьма нестабильным. Нестабильное поведение стохастической модификации на начальных итерациях хорошо заметно на графике с нормой градиента. Как правило, для более сложных моделей наблюдаются ещё большие колебания. Путём подбора величины шага можно добиться лучшей скорости сходимости, и существуют методы, адаптивно подбирающие величину шага (например, AdaGrad, Adam, RMSProp).

### Итог

Градиентный спуск -- очень мощный инструмент для решения задач машинного обучения. Возможно, это наиболее широко применимый в этой области алгоритм. 

__Дополнительно.__ Обновляемая статья, в которой собраны наиболее актуальные модификации алгоритма градиентного спуска: https://www.ruder.io/optimizing-gradient-descent/

